In [8]:
from audio_preprocessing.cut_audio_segment_from_audio_file import AudioProcessor
from feature_extraction.feature_extractor import FeatureExtractor
from feature_extraction.mfcc_images_extractor import MelSpectrogramImageExtractor
from data_processing.english_data_processing import ModelEvaluator
from itertools import combinations
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

In [9]:
def extract_mfcc_images(n_mfcc, figsize, input_folder, output_folder):
    mfccImageExtractor = MelSpectrogramImageExtractor(n_mfcc, figsize)
    mfccImageExtractor.extract_from_folders(input_folder, output_folder)

In [10]:
def cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, input_folder, output_folder):
    segmentation = AudioProcessor(frame_size, hop_size, threshold)
    segmentation.process_folders(input_folder, output_folder)

In [11]:
def extract_feature(n_mfcc, input_folder, output_folder, selected_features):
    featureExtractor = FeatureExtractor(n_mfcc)
    featureExtractor.process_folder(input_folder, output_folder, selected_features)
    # featureExtractor.process_folder(input_folder, output_folder, ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

In [12]:
def model(featured_data):
    models = [RandomForestClassifier, ExtraTreesClassifier]
    
    df_train_test = pd.read_csv(featured_data)
    df_shuffled = df_train_test.sample(frac=1, random_state=42).reset_index(drop=True)

    modelEvaluator = ModelEvaluator(models)
    best_model, best_scaler, best_avg_acc = modelEvaluator.evaluate_models(df_shuffled)
    return best_model, best_scaler, best_avg_acc

In [13]:
def process_data(frame_size, hop_size, threshold, raw_folder, cutted_folder, n_mfcc_list, featured_data_path, n_mfcc_for_images, images_folder, features):
    cut_audio_segment_from_audio_file(frame_size, hop_size, threshold, raw_folder, cutted_folder)
    n = len(features)

    results = []

    for n_mfcc in n_mfcc_list:
        for r in range(0, n + 1):
            for combo in combinations(range(n), r):
                selected_features = [features[i] for i in range(n) if i not in combo]
                extract_feature(n_mfcc, cutted_folder, featured_data_path, selected_features)
                current_model, current_scaler, current_avg_acc = model(featured_data_path)
                print(f"Scaler: {current_scaler}, Model: {current_model}, n_mfcc: {n_mfcc}, Selected features: {selected_features}, Accuracy: {current_avg_acc}")
                results.append({'Scaler': current_scaler, 'Model': current_model, 'N_MFCC': n_mfcc, 'Selected Features': selected_features, 'Average Accuracy': current_avg_acc})

    results_df = pd.DataFrame(results)
    ranked_results = results_df.sort_values(by='Average Accuracy', ascending=False)

    return ranked_results

In [14]:
process_data(256, 128, 0.0005, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/combined_data', 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data0', [13,20,26,40], 'D:/data_analysis/speech_emotion_recognition/notebooks/feature2.csv', 100, 'D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/images', ['mfcc','mfcc_statistics', 'zcr', 'pitch', 'rms'])

d:\data_analysis\speech_emotion_recognition\src\audio_preprocessing\utils\audio_utils.py:79: RuntimeWarning: invalid value encountered in divide
  return ste / np.max(ste)
2024-05-22 14:25:15.261 | INFO     | feature_extraction.feature_extractor:process_folder:148 - Processing input folder: D:/data_analysis/speech_emotion_recognition/data/EnglishDataset/cleaned_data0
2024-05-22 14:28:27.663 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature2.csv
2024-05-22 14:28:27.758 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 14:29:34.615 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6168331269094212
2024-05-22 14:29:50.054 | INFO     | data_processing.english_data_processing:evaluate_models:74

Scaler: MinMaxScaler, Model: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, n_mfcc: 13, Selected features: ['mfcc', 'mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6244680868631823


2024-05-22 14:38:33.550 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature2.csv
2024-05-22 14:38:33.631 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 14:39:30.763 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.6231932953513335
2024-05-22 14:39:44.104 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.6176504004623896
2024-05-22 14:40:44.531 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.6189234580133762
2024-05-22 14:40:57.958 | INFO     | data

Scaler: None, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 13, Selected features: ['mfcc_statistics', 'zcr', 'pitch', 'rms'], Accuracy: 0.6231932953513335


2024-05-22 14:48:04.023 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature2.csv
2024-05-22 14:48:04.058 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 14:48:38.780 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.5883865081331021
2024-05-22 14:48:52.820 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.5912955990421931
2024-05-22 14:49:35.130 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.5912941127900255
2024-05-22 14:49:48.145 | INFO     | data

...

Scaler: RobustScaler, Model: <class 'sklearn.ensemble._forest.RandomForestClassifier'>, n_mfcc: 13, Selected features: ['mfcc_statistics'], Accuracy: 0.6125600693584344


2024-05-22 17:55:32.744 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature2.csv
2024-05-22 17:55:32.767 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....
2024-05-22 17:55:57.769 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: None, accuracy: 0.5674838576500703
2024-05-22 17:56:05.962 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> - scaler: None, accuracy: 0.5711189827429608
2024-05-22 17:56:30.166 | INFO     | data_processing.english_data_processing:evaluate_models:74 - Model <class 'sklearn.ensemble._forest.RandomForestClassifier'> - scaler: StandardScaler(), accuracy: 0.5669368342828834
2024-05-22 17:56:38.368 | INFO     | data

Scaler: MaxAbsScaler, Model: <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, n_mfcc: 13, Selected features: ['mfcc'], Accuracy: 0.5755707208323012


2024-05-22 17:58:50.926 | INFO     | feature_extraction.feature_extractor:process_folder:158 - Feature extraction completed. Data saved to: D:/data_analysis/speech_emotion_recognition/notebooks/feature2.csv
2024-05-22 17:58:50.937 | INFO     | data_processing.english_data_processing:evaluate_models:53 - Finding best model....


ValueError: at least one array or dtype is required